In [38]:
import json
import pandas as pd

In [39]:
data_df = pd.read_csv('merged_data_1.csv')
len(data_df)


1532

In [40]:
data_df.head()

,review_id,product_id,title,author,rating,content,timestamp,profile_id,is_verified,helpful_count,product_attributes
0,R3ED0R4VCG7DP7,B0CXL4FQBK,5.0 out of 5 stars Amazing Speaker for All Needs,Colin,5,I was in need of a new versatile portable spea...,"Reviewed in the United States June 24, 2024",AHWLZPJ4HLRGSDCGXHCEWVVOGX4Q,TRUE,17,Color: Off White
1,R7FS5ZX9N85YV,B0CXL4FQBK,5.0 out of 5 stars Concert Ready! Nice For the...,Melly Mel3,5,Color is as pictured. So pretty and vibrant.Ch...,"Reviewed in the United States August 3, 2024",AEQS3DLWRAFDJBBHMWTKH46UBBDQ,TRUE,0,Color: Orange
2,R3MI1MH02YEDHK,B0CXL4FQBK,5.0 out of 5 stars Things I wish I knew before...,Pat,5,"The media could not be loaded. So, yes 5 stars...","Reviewed in the United States May 20, 2024",AGCVXRAC7KRNWUCTJSFA2D44CDNA,TRUE,51,Color: Forest Gray
3,R26O4F9FKNIYVG,B0CXL4FQBK,4.0 out of 5 stars Easy to pair.,Joseph C,4,Not a Bose flex link. But good for the price. ...,"Reviewed in the United States July 6, 2024",AEWWQRYIV3XHR6HRM6YALMVX6DPQ,TRUE,2,Color: Black
4,R1SSV73W8JIH5E,B0CXL4FQBK,5.0 out of 5 stars Didn't expect the sound qua...,Mike Freeman,5,"After trying out Sony Wireless Speaker, I have...","Reviewed in the United States May 10, 2024",AFKMNV7MVRWKIXFNFEJOIYWBOUSA,TRUE,13,Color: Off White


In [41]:
#dropping duplicates
data_df = data_df.drop_duplicates(subset=['review_id', 'author', 'product_id'], keep='first')
len(data_df)

1240

In [42]:
#Dropping data with no reviews
data_df.dropna(subset=['content'], inplace=True)

#replacing null values
data_df = data_df.fillna('not listed')

#casting data type
data_df['author'] = data_df['author'].astype(str)
data_df['timestamp'] = data_df['timestamp'].astype(str)
data_df[['review_id', 'product_id', 'title', 'content', 'profile_id', 'product_attributes']] = data_df[['review_id', 'product_id', 'title', 'content', 'profile_id', 'product_attributes']].astype(str)
data_df['rating'] = data_df['rating'].astype(int)
data_df['helpful_count'] = data_df['helpful_count'].astype(int)
data_df['is_verified'] = data_df['is_verified'].astype(bool)


In [43]:
#extracting date from timestamp
data_df['date'] = data_df['timestamp'].str.extract(r'(\w+ \d{1,2}, \d{4})')
data_df['date'] = pd.to_datetime(data_df['date'])
data_df.head()

,review_id,product_id,title,author,rating,content,timestamp,profile_id,is_verified,helpful_count,product_attributes,date
0,R3ED0R4VCG7DP7,B0CXL4FQBK,5.0 out of 5 stars Amazing Speaker for All Needs,Colin,5,I was in need of a new versatile portable spea...,"Reviewed in the United States June 24, 2024",AHWLZPJ4HLRGSDCGXHCEWVVOGX4Q,True,17,Color: Off White,2024-06-24
1,R7FS5ZX9N85YV,B0CXL4FQBK,5.0 out of 5 stars Concert Ready! Nice For the...,Melly Mel3,5,Color is as pictured. So pretty and vibrant.Ch...,"Reviewed in the United States August 3, 2024",AEQS3DLWRAFDJBBHMWTKH46UBBDQ,True,0,Color: Orange,2024-08-03
2,R3MI1MH02YEDHK,B0CXL4FQBK,5.0 out of 5 stars Things I wish I knew before...,Pat,5,"The media could not be loaded. So, yes 5 stars...","Reviewed in the United States May 20, 2024",AGCVXRAC7KRNWUCTJSFA2D44CDNA,True,51,Color: Forest Gray,2024-05-20
3,R26O4F9FKNIYVG,B0CXL4FQBK,4.0 out of 5 stars Easy to pair.,Joseph C,4,Not a Bose flex link. But good for the price. ...,"Reviewed in the United States July 6, 2024",AEWWQRYIV3XHR6HRM6YALMVX6DPQ,True,2,Color: Black,2024-07-06
4,R1SSV73W8JIH5E,B0CXL4FQBK,5.0 out of 5 stars Didn't expect the sound qua...,Mike Freeman,5,"After trying out Sony Wireless Speaker, I have...","Reviewed in the United States May 10, 2024",AFKMNV7MVRWKIXFNFEJOIYWBOUSA,True,13,Color: Off White,2024-05-10


In [44]:
#cleaning product_attributes column
def extract_attributes(attribute_string):
    attributes = {}

    if 'Color: ' in attribute_string:
        color_part = attribute_string.split('Color: ')[-1]
        attributes['Color'] = color_part.split('Style: ')[0].strip() if 'Style: ' in color_part else color_part.strip()

    else:
        attributes['Color'] = None


    if 'Style: ' in attribute_string:
        style_part = attribute_string.split('Style: ')[-1]
        attributes['Style'] = style_part.split('Pattern: ')[0].strip() if 'Pattern: ' in style_part else style_part.strip()
        #attributes['Pattern'] = style_part.split('Pattern: ')[-1].strip() if 'Pattern: ' in style_part else None
    else:
        attributes['Style'] = None
        #attributes['Pattern'] = None

    return attributes

attributes_df = data_df['product_attributes'].apply(extract_attributes).apply(pd.Series)

data_df = pd.concat([data_df, attributes_df], axis=1)
data_df = data_df.fillna('not listed')

print(data_df)

           review_id  product_id  \
0     R3ED0R4VCG7DP7  B0CXL4FQBK   
1      R7FS5ZX9N85YV  B0CXL4FQBK   
2     R3MI1MH02YEDHK  B0CXL4FQBK   
3     R26O4F9FKNIYVG  B0CXL4FQBK   
4     R1SSV73W8JIH5E  B0CXL4FQBK   
...              ...         ...   
1527  R3CD2I4S2X97Z0  B097XX34SL   
1528  R3AZLYB5EPJ8WG  B097XX34SL   
1529  R3CPR2M7G6UAPQ  B097XX34SL   
1530   RHMJU0AYUY3AH  B097XX34SL   
1531   R6F4QFJZUBYF1  B097XX34SL   

                                                  title                author  \
0      5.0 out of 5 stars Amazing Speaker for All Needs                 Colin   
1     5.0 out of 5 stars Concert Ready! Nice For the...            Melly Mel3   
2     5.0 out of 5 stars Things I wish I knew before...                   Pat   
3                      4.0 out of 5 stars Easy to pair.              Joseph C   
4     5.0 out of 5 stars Didn't expect the sound qua...          Mike Freeman   
...                                                 ...                   ...   


In [46]:
len(data_df)

1238

In [47]:
#Outlier data in dataset --> problems with csv formatting
data_df = data_df[data_df['Pattern'] != 'Speaker']
len(data_df)

1218

In [48]:
data_df.to_csv('amazon_data_final.csv', index=False)
